In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("SQL_GlobalSparkSession").getOrCreate()


24/03/09 22:45:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
# 💻 Write a pyspark code to convert table A value as table B.
from pyspark.sql.functions import col,expr

schema=['key','values']
data= [('K1',[1,1,2]),('K2',[1,2,3])]
df1=spark.createDataFrame(data,schema)
df1.createOrReplaceTempView("keyvalues")
df1.select(col('key'),expr("values[0]").alias("A"),expr("values[1]").alias("B"), expr("values[2]").alias("C")).show()
spark.sql("""
    select key,values[0] as A,values[1] as B,values[2] as C from keyvalues
""").show()

In [ ]:
"""𝐏𝐫𝐨𝐛𝐥𝐞𝐦 𝐒𝐭𝐚𝐭𝐞𝐦𝐞𝐧𝐭 :
Write a query that'll identify returning active users. 
A returning active user is a user that has made a second purchase within 7 days of any other of their purchases.
Output a list of user_ids of these returning active users.
"""
from pyspark.sql.functions import col
from pyspark.sql.types import *

schema=['user_id','item','created_at','revenue']

data=[ (100, 'bread', '07-03-2020', 410),(100, 'banana' ,'13-03-2020' ,175),
        (100, 'banana','29-03-2020', 599),(101, 'milk', '01-03-2020', 449),
        (101, 'milk', '26-03-2020', 740),(114, 'banana', '10-03-2020', 200),
        (114, 'biscuit', '16-03-2020', 300)
     ]

df1=spark.createDataFrame(data,schema)
# df1.show()
df1.createOrReplaceTempView("revenues_2")

spark.sql("""
with clean_data as(
    select user_id,item,revenue
    ,cast(substr(created_at,7,4)||'-'||substr(created_at,4,2)||'-'||substr(created_at,1,2) as date) as order_dt
    from revenues_2
)
--FInd frequency of purchase
,find_frequency as (
    select user_id,item,revenue,order_dt
    --,coalesce(lag(order_dt) over(partition by user_id order by order_dt) ,current_date()) as prev_date
    ,datediff(order_dt,lag(order_dt) over(partition by user_id order by order_dt)) as diff_days
    from clean_data
)
select * 
from find_frequency
where diff_days <= 7
""")#.show()
# ans 100,114

spark.sql("""
select date('07-03-2020') as dt
""").show()

In [ ]:
"""𝐏𝐫𝐨𝐛𝐥𝐞𝐦 𝐒𝐭𝐚𝐭𝐞𝐦𝐞𝐧𝐭 :
Given a table of purchases by date, calculate the month-over-month 𝐩𝐞𝐫𝐜𝐞𝐧𝐭𝐚𝐠𝐞 𝐜𝐡𝐚𝐧𝐠𝐞 𝐢𝐧 𝐫𝐞𝐯𝐞𝐧𝐮𝐞.
The output should include the year-month date (YYYY-MM) and percentage change, rounded to the 2nd decimal point,
and sorted from the beginning of the year to the end of the year.
"""
schema=['id','created_at','value','purchase_id']

data=[(1,'01-01-2019',172692,43), (2,'05-01-2019',177194,36),
        (3,'06-02-2019',116948,56), (4,'10-02-2019',162515,29),
        (5,'14-02-2019',120741,30), (6,'22-03-2019',151688,34),
        (7,'26-03-2019',1002327,44)
     ]

df1=spark.createDataFrame(data,schema)
# df1.show()
df1.createOrReplaceTempView("revenues")

spark.sql("""
with sum_amt (
    select
     substring(created_at,4,10) as year_month
    ,sum(value) as curr_month
    from revenues
    group by substring(created_at,4,10)
    )
,get_prevmonth as(
    select year_month,curr_month
        ,coalesce(lag(curr_month) over(order by year_month),0) as prev_month
        from sum_amt
    )
select year_month,curr_month,prev_month
    ,case when prev_month !=0 then round((curr_month-prev_month)/prev_month * 100,2)
    else null end as revenue_percentage
    from get_prevmonth
    
    """).show()

In [ ]:
""" 𝐏𝐫𝐨𝐛𝐥𝐞𝐦 𝐒𝐭𝐚𝐭𝐞𝐦𝐞𝐧𝐭 :
Write an SQL query to find how many users visited the bank and didn't do any transactions,
how many visited the bank and did one transaction and so on.
"""

visits_schema=['user_id','visit_date']
transactions_schema=['user_id','transaction_date','amount']

visits=[('1', '2020-01-01'),('2', '2020-01-02'),('12', '2020-01-01'),('19', '2020-01-03')
        ,('1', '2020-01-02'),('2', '2020-01-03'),('1', '2020-01-04'),('7', '2020-01-11'),
        ('9', '2020-01-25'),('8', '2020-01-28')]

transactions=[('1', '2020-01-02', '120'), ('2', '2020-01-03', '22'),('7', '2020-01-11', '232'),('1', '2020-01-04', '7')
             ,('9', '2020-01-25', '33'),('9', '2020-01-25', '66'),('8', '2020-01-28', '1'),('9', '2020-01-25', '99')
             ]
df1=spark.createDataFrame(visits,visits_schema)
# df1.show()
df1.createOrReplaceTempView("visits")
df2=spark.createDataFrame(transactions,transactions_schema)
# df2.show()
df2.createOrReplaceTempView("transactions")
spark.sql("""
with find_0 as (
    select 
    0 as t_count,
    count(v.user_id) as v_count
    
    from visits as v
    left join transactions as t
    on v.user_id=t.user_id and v.visit_date=t.transaction_date
    where t.user_id is null
)
,find_13 as(
select t_count as no_of_trans,count(1) as t_count 
from(
    select t.user_id,t.transaction_date,count(1) as t_count
    from transactions as t
    group by t.user_id,t.transaction_date
)
group by t_count
)
select * from find_0
union
select * from find_13
""")#.show()

In [ ]:
"""
#thoughtworks SQL Interview Question for Data Engineer , Data Analyst and Data Science role.

𝐏𝐫𝐨𝐛𝐥𝐞𝐦 𝐒𝐭𝐚𝐭𝐞𝐦𝐞𝐧𝐭 : A 𝐟𝐫𝐢𝐞𝐧𝐝𝐬𝐡𝐢𝐩 between a pair of friends x and y is strong if x and y have at least three common friends.
Write an SQL query to find all the strong friendships.
Note that the result table should not contain duplicates with user1_id < user2_id ( To have the friendship user1_id < user2_id)
"""

schema=['user1_id','user2_id']
data=[('1', '2'),('1', '3'),('2', '3'),('1', '4'),('2', '4'),('1', '5')
      ,('2', '5'),('1', '7'), ('3', '7'), ('1', '6'),('3', '6'),('2', '6')]
df=spark.createDataFrame(data,schema)
# df.show()
df.createOrReplaceTempView("Friendship")

spark.sql("""
SELECT DISTINCT
    F1.user1_id,
    F1.user2_id
FROM
    Friendship F1
JOIN
    Friendship F2 ON F1.user1_id = F2.user1_id AND F1.user2_id = F2.user2_id AND F1.user1_id < F1.user2_id
JOIN
    Friendship F3 ON F1.user1_id = F3.user1_id AND F1.user2_id = F3.user2_id AND F1.user1_id < F3.user1_id AND F1.user2_id < F3.user2_id
    AND F2.user1_id = F3.user1_id AND F2.user2_id = F3.user2_id
ORDER BY
    F1.user1_id,
    F1.user2_id""").show()

In [ ]:
""" Pending
𝐏𝐫𝐨𝐛𝐥𝐞𝐦 𝐒𝐭𝐚𝐭𝐞𝐦𝐞𝐧𝐭:
𝐘𝐨𝐮 𝐚𝐫𝐞 𝐠𝐢𝐯𝐞𝐧 𝐭𝐡𝐞 𝐩𝐫𝐢𝐜𝐞 𝐨𝐟 𝐞𝐚𝐜𝐡 𝐬𝐤𝐮 𝐰𝐡𝐞𝐧𝐞𝐯𝐞𝐫 𝐭𝐡𝐞𝐫𝐞 𝐢𝐬 𝐚 𝐜𝐡𝐚𝐧𝐠𝐞 𝐢𝐧 𝐩𝐫𝐢𝐜𝐞.
Write an SQL query 𝐭𝐨 𝐟𝐢𝐧𝐝 𝐭𝐡𝐞 𝐩𝐫𝐢𝐜𝐞 𝐚𝐭 𝐭𝐡𝐞 𝐬𝐭𝐚𝐫𝐭 𝐨𝐟 𝐞𝐚𝐜𝐡 𝐦𝐨𝐧𝐭𝐡 & 𝐜𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐝𝐢𝐟𝐟𝐞𝐫𝐞𝐧𝐜𝐞 𝐟𝐫𝐨𝐦 𝐭𝐡𝐞 𝐩𝐫𝐞𝐯𝐢𝐨𝐮𝐬 𝐦𝐨𝐧𝐭𝐡'𝐬 𝐬𝐭𝐚𝐫𝐭 𝐝𝐚𝐭𝐞.
"""
schema=['sku_id','price_date','price']

data = [(1,'2023-01-01',10)
,(1,'2023-02-15',15)
,(1,'2023-03-03',18)
,(1,'2023-03-27',15)
,(1,'2023-04-06',20)]
df=spark.createDataFrame(data,schema)
# df.show()
df.createOrReplaceTempView("sku")

spark.sql("""
    select *--,price-prev_price as price_diff 
    from(
    select *,coalesce(lag(price) over(partition by sku_id order by price_date ),0) as prev_price
    ,price-coalesce(lag(price) over(partition by sku_id order by price_date ),0) as prev_pricediff
    from sku --order by price
    )
""")
#.show()

In [ ]:
"""
Find the stores which either sell both tea and coffee  or coffee and jam
"""

schema=['store_id','store_name','product']
data=[
(1,'bb','tea'),
(1,'bb','coffee'),
(2,'dmart','coffee'),
(3,'more','tea'),
(4,'reliance','tea'),
(4,'reliance','coffee'),
(4,'reliance','jam'),
(5,'b mart','coffee'),
(5,'b mart','jam')
]
df=spark.createDataFrame(data,schema)
# df.show()
df.createOrReplaceTempView("stores")

spark.sql("""
select store_id,store_name--,product
,max(case when product='tea' then product else null end) as tea
,max(case when product='coffee' then product else null end) as coffee
,max(case when product='jam' then product else null end) as jam

from stores
group by store_id,store_name
having (tea='tea' and coffee='coffee') or (jam='jam' and coffee='coffee')

""")
spark.sql(""" 
    SELECT store_id, store_name
    FROM stores
    WHERE product IN ('tea', 'coffee', 'jam')
    GROUP BY store_id, store_name
    HAVING COUNT(DISTINCT product) >= 2
""")# .show()

In [ ]:
'''Asked in indiumsoft L2 Round
return the customers who place the order within 12 days
'''
data=[
(1,'2021/12/31','123a','b1'),
(2,'2022/01/11','123a','b2'),
(3,'2023/01/01','123a','b3'),
(4,'2023/02/02','123a','b4'),
(5,'2023/01/01','346b','b5'),
(6,'2023/02/02','346b','b6'),
(7,'2023/02/14','346b','b7'),
(8,'2023/02/17','346b','b7')
]
schema=['Orderid','Orderdt','custid','Endloc']

df=spark.createDataFrame(data,schema)
df.createOrReplaceTempView("orders")
# way 1
spark.sql("""
  select *,datediff(orderdt,prevdt) as days
    from (
        select Orderid,to_date(Orderdt,'yyyy/MM/dd') as orderdt
            ,custid,Endloc
            ,coalesce(lag(to_date(Orderdt,'yyyy/MM/dd')) over(partition by custid order by custid),orderdt ) as prevdt
        from orders 
        order by custid,orderdt
        )
        where datediff(orderdt,prevdt) <=12
""")#.show()

# way 2
spark.sql("""
select * from (
select *
--,datediff(orderdt,lag(orderdt) over(partition by custid order by custid) ) as days
,to_date(orderdt,'yyyy/mm/dd') as currentdt, lag(to_date(orderdt,'yyyy/mm/dd')) over(partition by custid order by custid) as prevdt

from orders )
--where days <=12
""")\
# .show()

# print(type(df.Orderdt))
# df.show()

In [ ]:
"""Asked in synechron
Metro Train table is ordered by turn in the example for simplicity.
In the example Rajkumar(id 105). Shruthitid 103) and Ravikiran (id 106) will enter 
the Metro_Train as their weight sum is 250-350+400=1000
Ravikiran (id/106) is the last person to fir in the Metro Train because he has the last turn in these three people.
"""
schema=['id','name','weight','turn_id']
data=[(105,'Rajkumar',250,1)
,(103,'Shruthi',350,2)
,(106,'Ravikiran',400,3)
,(102,'Tejuswini',200,4)
,(104,'Ravikiran',1175,5)
,(101,'Surya',500,6)]



In [ ]:
Name some of the frameworks that SCALA supports
how to skip first few rows from a datafile in pyspark
ETL vs ELT
sheedusele single mode duster with 64dores and 512/68 memory.or should go for smaller nodes such as eight of 8 cores and 64 GB memory?
Which one is better in which scenario?

In [ ]:
"""
𝑪𝒉𝒂𝒍𝒍𝒆𝒏𝒈𝒆 : Get the name of the 𝐡𝐢𝐠𝐡𝐞𝐬𝐭 𝐬𝐚𝐥𝐚𝐫𝐲 𝐚𝐧𝐝 𝐥𝐨𝐰𝐞𝐬𝐭 𝐬𝐚𝐥𝐚𝐫𝐲 employee name in 𝐞𝐚𝐜𝐡 𝐝𝐞𝐩𝐚𝐫𝐭𝐦𝐞𝐧𝐭.
If the salary is same then return the name of the employee whose name comes first in 𝐥𝐞𝐱𝐢𝐜𝐨𝐠𝐫𝐚𝐩𝐡𝐢𝐜𝐚𝐥 𝐨𝐫𝐝𝐞𝐫.
"""
from pyspark.sql.functions import *

emp_data = [
    ('Siva',1,30000),('Ravi',2,40000),('Prasad',1,50000),
    ('Arun',1,30000),('Sai',2,20000)
]

emp_schema = "emp_name string , dep_id int , salary long"

# Creating the dataframe
df = spark.createDataFrame(data = emp_data , schema = emp_schema).createOrReplaceTempView("depts")

spark.sql("""
select dep_id
    ,max((case when minSal=1 then emp_name end)) as minEmp
    ,max((case when maxSal=1 then emp_name end)) as maxEmp
    from ( select dep_id,emp_name,salary
    ,row_number() over(partition by dep_id order by salary,emp_name) as minSal
    ,row_number() over(partition by dep_id order by salary desc,emp_name) as maxSal
    from depts 
    )
    group by dep_id
""").show()